<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c32815c157000f24fe3bdb682bdd1218a4fed9cf793ea4d2c61455efce98308f
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-24 14:23:19
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.40 C
-------------------
Today PnL: -48.61 K (-0.35%)
Current PnL: -19.46 L (-13.09%)
CY Booked + Current PnL: -8.02 L (-5.4%)
-------------------
Total profit:  1.47 L
Total loss:  -20.93 L
-------------------
Total Booked + Current PnL: 18.96 L (15.58%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.16%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 85.33 L (60.89%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.62%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2080.00,12.54,57.0,H-MC,3.77,194210.0,29540.0,13789.0,-1.12,17.94,7.10,26.31,89.0,2.14,1.39,27.35,X40N,NTT,AC
77,TTKPRESTIG,770.00,96.37,39.0,M-SC,2.93,84626.0,-16151.0,16248.0,-0.52,-16.03,19.20,0.09,245.0,-0.99,0.60,11.18,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,58.0,H-MC,9.86,110033.0,9529.0,19575.0,-0.59,9.48,17.79,28.96,79.0,0.49,0.79,46.67,MH,ATH,METALS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.46,216481.0,6115.0,22666.0,-0.15,2.91,10.47,13.68,37.0,0.27,1.55,19.91,XY25,NTT,REFINERIES
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.97,205800.0,14058.0,23708.0,-0.05,7.33,11.52,19.69,99.0,0.59,1.47,14.63,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,53.51,67.0,H-SC,1.03,139903.0,-39168.0,104899.0,2.70,-21.87,74.98,36.71,135.0,-0.37,1.00,38.81,XR,NTT,BANKS
71,TANLA,1963.11,-14.58,73.0,H-SC,15.46,229102.0,-41659.0,371603.0,1.66,-15.39,162.20,121.86,133.0,-0.11,1.64,79.22,AR,ATH,IT
31,HINDUNILVR,2922.00,-8.70,46.0,H-LC,10.01,256380.0,-683.0,35816.0,1.65,-0.27,13.97,13.67,24.0,-0.02,1.83,20.09,XY25,NTT,FMCG
68,STARHEALTH,761.00,11.71,61.0,H-SC,4.85,248989.0,-32134.0,165005.0,1.52,-11.43,66.27,47.26,144.0,-0.19,1.78,33.40,XY24,NTT,INSURANCE
21,COFFEEDAY,80.00,-41.65,54.0,L-SC,42.85,86688.0,-26861.0,66914.0,1.44,-23.66,77.19,35.27,268.0,-0.40,0.62,112.17,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1217.38,72.0,M-SC,9.85,179099.0,-2044.0,47515.0,-3.14,-1.13,26.53,25.10,235.0,-0.04,1.28,43.53,XY24,NTT,HEALTHCARE
8,AWL,485.00,-65.97,45.0,H-MC,3.41,241949.0,-59802.0,221214.0,-2.60,-19.82,91.43,53.50,116.0,-0.27,1.73,7.47,XY24,NTT,FMCG
56,RELAXO,1176.00,-38.81,36.0,H-SC,2.18,83472.0,-61688.0,137612.0,-2.46,-42.50,164.86,52.31,136.0,-0.45,0.60,11.73,X40N,NTT,FOOTWEAR
46,KPIGREEN,731.05,-1.34,33.0,H-SC,9.54,117457.0,-7840.0,66763.0,-2.38,-6.26,56.84,47.03,141.0,-0.12,0.84,47.07,MH,ATH,POWER
26,GREENPANEL,537.00,210.72,47.0,M-SC,0.79,138592.0,-44486.0,123458.0,-2.34,-24.30,89.08,43.14,230.0,-0.36,0.99,30.36,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.22,43.0,H-MC,2.00,135510.0,-278.0,43024.0,-0.21,-0.20,31.75,31.48,107.0,-0.01,0.97,11.89,X40,ATH,INSURANCE
86,WIPRO,420.0,-13.34,42.0,M-LC,5.38,151912.0,967.0,108480.0,-1.85,0.64,71.41,72.51,53.0,0.01,1.08,6.70,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,92.53,50.0,H-SC,9.61,121608.0,-20871.0,40386.0,-0.38,-14.65,33.21,13.70,163.0,-0.52,0.87,43.53,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,96.37,39.0,M-SC,2.93,84626.0,-16151.0,16248.0,-0.52,-16.03,19.20,0.09,245.0,-0.99,0.60,11.18,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.11,220491.0,-49176.0,85506.0,-0.50,-18.24,38.78,13.47,138.0,-0.58,1.57,11.52,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-22.37,42.0,H-MC,6.90,105032.0,-26803.0,69027.0,-0.50,-20.33,65.72,32.03,98.0,-0.39,0.75,17.64,OX40N,NTT,IT
66,SIS,528.0,2085.49,39.0,H-SC,4.68,88192.0,-22840.0,46451.0,0.19,-20.57,52.67,21.26,156.0,-0.49,0.63,19.16,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,50.76,72.0,M-MC,9.80,237636.0,12674.0,154986.0,0.51,5.63,65.22,74.53,192.0,0.08,1.70,39.70,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,12.54,57.0,H-MC,3.77,194210.0,29540.0,13789.0,-1.12,17.94,7.10,26.31,89.0,2.14,1.39,27.35,X40N,NTT,AC
84,VOLTAS,1530.00,-1.92,46.0,H-MC,2.97,205800.0,14058.0,23708.0,-0.05,7.33,11.52,19.69,99.0,0.59,1.47,14.63,XY25,NTT,AC
17,CAMS,5211.76,-3.61,50.0,H-SC,2.25,109486.0,7482.0,36448.0,-1.90,7.34,33.29,43.06,122.0,0.21,0.78,25.71,X40N,ATH,MISC
23,DABUR,735.00,-7.08,42.0,H-MC,6.85,251739.0,3671.0,104723.0,0.79,1.48,41.60,43.70,102.0,0.04,1.80,16.14,XY24,BTT,FMCG


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,58.0,H-MC,9.86,110033.0,9529.0,19575.0,-0.59,9.48,17.79,28.96,79.0,0.49,0.79,46.67,MH,ATH,METALS
85,WHIRLPOOL,2270.00,-42.31,41.0,M-SC,1.42,97425.0,5926.0,72825.0,-1.66,6.48,74.75,86.07,223.0,0.08,0.70,41.48,XR,NTT,DURABLES
11,BANKINDIA,190.00,-26.71,70.0,H-MC,7.46,187544.0,7736.0,104668.0,0.42,4.30,55.81,62.52,88.0,0.07,1.34,39.86,XR,NTT,BANKS
37,INDIAMART,4850.92,-54.83,26.0,H-SC,10.71,126034.0,2698.0,131063.0,-0.54,2.19,103.99,108.45,119.0,0.02,0.90,25.03,AR,ATH,MISC
86,WIPRO,420.00,-13.34,42.0,M-LC,5.38,151912.0,967.0,108480.0,-1.85,0.64,71.41,72.51,53.0,0.01,1.08,6.70,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-19.59,33.0,H-LC,7.53,289524.0,-26318.0,138972.0,-1.03,-8.33,48.00,35.67,5.0,-0.19,2.07,4.37,X40N,ATH,FMCG
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196382.0,-3756.0,24646.0,-0.40,-1.88,12.55,10.44,4.0,-0.15,1.40,3.58,X40,NTT,FMCG
44,JIOFIN,387.00,-2.28,40.0,H-LC,13.26,265568.0,-1797.0,67242.0,-1.20,-0.67,25.32,24.48,48.0,-0.03,1.90,53.93,XY24,BTT,FINANCE
74,TCS,4476.75,-28.36,41.0,H-LC,12.12,286051.0,-59905.0,134759.0,-0.63,-17.32,47.11,21.64,1.0,-0.44,2.04,1.34,X40,ATH,IT
5,ASIANPAINT,3465.66,-14.11,43.0,H-LC,7.57,216691.0,-35077.0,88280.0,0.63,-13.93,40.74,21.13,27.0,-0.40,1.55,16.56,X40,ATH,PAINTS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-5.49,46.0,H-LC,1.54,234540.0,-11611.0,90814.0,-0.96,-4.72,38.72,32.18,16.0,-0.13,1.67,29.63,X200,ATH,IT
42,ITC,452.0,-39.19,39.0,H-LC,2.07,196382.0,-3756.0,24646.0,-0.40,-1.88,12.55,10.44,4.0,-0.15,1.40,3.58,X40,NTT,FMCG
65,SIEMENS,4671.5,-0.04,51.0,H-LC,3.08,160750.0,-25345.0,72820.0,-0.78,-13.62,45.30,25.51,15.0,-0.35,1.15,18.46,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-13.66,47.0,H-LC,4.46,216481.0,6115.0,22666.0,-0.15,2.91,10.47,13.68,37.0,0.27,1.55,19.91,XY25,NTT,REFINERIES
51,NESTLEIND,1377.0,-10.13,48.0,H-LC,4.54,276518.0,11092.0,45708.0,1.14,4.18,16.53,21.40,11.0,0.24,1.97,11.57,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,41.0,H-LC,12.12,286051.0,-59905.0,134759.0,-0.63,-17.32,47.11,21.64,1.0,-0.44,2.04,1.34,X40,ATH,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196382.0,-3756.0,24646.0,-0.40,-1.88,12.55,10.44,4.0,-0.15,1.40,3.58,X40,NTT,FMCG
83,VBL,671.64,-19.59,33.0,H-LC,7.53,289524.0,-26318.0,138972.0,-1.03,-8.33,48.00,35.67,5.0,-0.19,2.07,4.37,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-11.09,60.0,H-LC,7.25,233026.0,-1118.0,49751.0,1.01,-0.48,21.35,20.77,86.0,-0.02,1.66,6.41,X40N,NTT,BREWERIES
30,HCLTECH,1943.91,-1.08,43.0,H-LC,9.91,226080.0,-15844.0,79105.0,0.71,-6.55,34.99,26.15,8.0,-0.20,1.61,6.84,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-0.04,51.0,H-LC,3.08,160750.0,-25345.0,72820.0,-0.78,-13.62,45.30,25.51,15.0,-0.35,1.15,18.46,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196382.0,-3756.0,24646.0,-0.40,-1.88,12.55,10.44,4.0,-0.15,1.40,3.58,X40,NTT,FMCG
19,CIPLA,1795.00,-20.81,46.0,H-LC,5.18,211131.0,6631.0,34794.0,0.92,3.24,16.48,20.25,10.0,0.19,1.51,12.26,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-14.11,43.0,H-LC,7.57,216691.0,-35077.0,88280.0,0.63,-13.93,40.74,21.13,27.0,-0.40,1.55,16.56,X40,ATH,PAINTS
57,RELIANCE,1533.00,-13.66,47.0,H-LC,4.46,216481.0,6115.0,22666.0,-0.15,2.91,10.47,13.68,37.0,0.27,1.55,19.91,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-28.36,41.0,H-LC,12.12,286051.0,-59905.0,134759.0,-0.63,-17.32,47.11,21.64,1.0,-0.44,2.04,1.34,X40,ATH,IT
40,INFY,2275.00,-18.52,47.0,H-LC,8.12,317711.0,4823.0,166862.0,-0.39,1.54,52.52,54.87,3.0,0.03,2.27,8.29,X40,BTT,IT
42,ITC,452.00,-39.19,39.0,H-LC,2.07,196382.0,-3756.0,24646.0,-0.40,-1.88,12.55,10.44,4.0,-0.15,1.40,3.58,X40,NTT,FMCG
83,VBL,671.64,-19.59,33.0,H-LC,7.53,289524.0,-26318.0,138972.0,-1.03,-8.33,48.00,35.67,5.0,-0.19,2.07,4.37,X40N,ATH,FMCG
1,ABB,7934.00,-39.28,52.0,H-LC,8.63,253608.0,-8011.0,127235.0,-0.29,-3.06,50.17,45.57,7.0,-0.06,1.81,7.91,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6850.00,54.0,L-SC,5.32,76282.0,-17528.0,94147.0,-1.35,-18.68,123.42,81.67,269.0,-0.19,0.54,47.72,XR,NTT,CERAMICS
61,SAMMAANCAP,326.00,-171.59,55.0,M-SC,4.89,81930.0,-20280.0,113670.0,1.13,-19.84,138.74,91.37,208.0,-0.18,0.58,33.00,XY25,NTT,FINANCE
21,COFFEEDAY,80.00,-41.65,54.0,L-SC,42.85,86688.0,-26861.0,66914.0,1.44,-23.66,77.19,35.27,268.0,-0.40,0.62,112.17,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-7.03,40.0,H-SC,11.74,86589.0,-14374.0,108903.0,-0.53,-14.24,125.77,93.63,148.0,-0.13,0.62,28.97,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,54.0,H-SC,12.69,94965.0,-3015.0,24615.0,0.68,-3.08,25.92,22.05,152.0,-0.12,0.68,38.04,XR,ATH,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1217.38,72.0,M-SC,9.85,179099.0,-2044.0,47515.0,-3.14,-1.13,26.53,25.10,235.0,-0.04,1.28,43.53,XY24,NTT,HEALTHCARE
71,TANLA,1963.11,-14.58,73.0,H-SC,15.46,229102.0,-41659.0,371603.0,1.66,-15.39,162.20,121.86,133.0,-0.11,1.64,79.22,AR,ATH,IT
7,ATULAUTO,844.00,3748.65,55.0,M-SC,5.97,120030.0,-27338.0,79160.0,-2.30,-18.55,65.95,35.16,236.0,-0.35,0.86,22.60,XY24,NTT,AUTO
60,SAIL,228.00,50.76,72.0,M-MC,9.80,237636.0,12674.0,154986.0,0.51,5.63,65.22,74.53,192.0,0.08,1.70,39.70,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,59.78,59.0,H-SC,5.97,138310.0,-44465.0,155530.0,-0.78,-24.33,112.45,60.77,125.0,-0.29,0.99,23.88,XR,NTT,JEWELLERY


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.29
1,25,43.77
2,50,74.82


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.95
LC    32.50
MC    23.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.29
X40      14.73
X40N     11.40
XR       11.30
XY25     10.10
AR        8.43
OX40N     7.81
X200      1.67
MH        1.63
X5K       1.43
SR        1.23
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.38
H-LC    25.99
H-MC    20.60
M-SC    13.05
M-LC     5.47
M-MC     2.67
L-SC     1.52
L-LC     1.04
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.91,-5.34,40.11
IT,12.98,-15.60,78.62
FINANCE,10.04,-16.66,67.56
MISC,6.98,-17.44,82.73
BANKS,6.18,-15.34,76.01
PAINTS,5.64,-16.75,34.15
ELECTRICAL,5.47,-7.46,45.16
INSURANCE,3.74,-5.69,45.20
AC,3.56,2.66,13.64


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3138195.0,22
XR,1326795.0,14
AR,1152842.0,9
X40,785053.0,10
X40N,609251.0,9
OX40N,552663.0,10
XY25,424167.0,7
SR,249844.0,2
X5K,116898.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3679997.0,29
M-SC,1416985.0,17
H-MC,1222950.0,15
H-LC,1179470.0,15
M-LC,383303.0,4
M-MC,304064.0,2
L-SC,247496.0,3
L-MC,60216.0,1
L-LC,38379.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1234520.0      6
M-SC       XY24       810851.0      7
H-SC       AR         786896.0      5
           XR         774471.0      7
H-MC       XY24       563593.0      4
H-LC       X40        493652.0      5
H-SC       X40N       305894.0      4
M-MC       XY24       304064.0      2
H-SC       OX40N      261609.0      4
           SR         249844.0      2
H-LC       X40N       223517.0      3
H-MC       X40        218319.0      4
H-LC       AR         200055.0      2
H-MC       XY25       167928.0      2
M-SC       AR         165891.0      2
L-SC       XR         161061.0      2
M-LC       XY24       157925.0      2
M-SC       OX40N      135592.0      4
           XR         117899.0      2
M-LC       X5K        116898.0      1
M-SC       XY25       113670.0      1
M-LC       XR         108480.0      1
H-MC       XR         104668.0      1
H-LC       XY25       104190.0      3
           X200        90814.0      1
L-SC       OX40N       86435.0      1
H-MC       X40N        79840.0      2
M-SC       X40         73082.0      1
H-MC       OX40N       69027.0      1
H-LC       XY24        67242.0      1
H-SC       MH          66763.0      1
L-MC       XR          60216.0      1
L-LC       XY25        38379.0      1
H-MC       MH          19575.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
